<a href="https://colab.research.google.com/github/thetachenlab/Neural-Network-Bootcamp/blob/main/MNIST_digit_recognizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('sample_data/mnist_train_small.csv')

In [5]:
from re import X
data = np.array(data)
m, n = data.shape
print(m,n)
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:1000].T

Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

print(data_train.shape)
print(X_train.shape)

19999 785
(785, 18999)
(784, 18999)



* ${X_j}= [784\times m]$ where $m$ is the number of training dataset.
* ${Y_j}= [10\times m]$ where $m$ is the number of training dataset.

First layer
* $W^{[1]}_{ij}= [10\times 784]$
* $B^{[1]}_i = [10\times 1]$
* ${Z^{[1]}_i}= [10\times m]$
* ${A^{[1]}_i}= [10\times m]$

Second layer
* $W^{[2]}_{ij}= [10\times10]$
* $B^{[2]}_i = [10\times 1]$
* ${Z^{[2]}_i}= [10\times m]$
* ${A^{[2]}_i}= [10\times m]$

Forward propagation:
\begin{align}
Z^{[1]}_i &= \sum_j^{784} W^{[1]}_{ij}X_j+B^{[1]}_i \\
A^{[1]}_i &= \text{ReLU} (Z^{[1]}_i) \\
Z^{[2]}_i &= \sum_j^{10} W^{[2]}_{ij}A^{[1]}_j+B^{[2]}_i \\
A^{[2]}_i &= \text{softmax} (\{Z^{[2]}_i\}) \\
\end{align}
where $\text{softmax}(\{Z^{[2]}_i\})=e^{Z^{[2]}_i}/\sum_j e^{Z^{[2]}_j}$.

Cross-entropy loss
\begin{equation}
\mathcal{L} = -\sum_i Y_i\log(A^{[2]}_i)
\end{equation}

Backward propagation
\begin{align}
\frac{\partial\mathcal{L}}{\partial A^{[2]}_i}=-\frac{Y_i}{A^{[2]}_i}
\end{align}
\begin{align}
\frac{\partial A^{[2]}_i}{\partial Z^{[2]}_j}&=A^{[2]}_i(1-A^{[2]}_i)\delta_{ij}-A^{[2]}_iA^{[2]}_j(1-\delta_{ij}) \\
\frac{\partial\mathcal{L}}{\partial Z^{[2]}_j}&=\sum_i\frac{\partial\mathcal{L}}{\partial A^{[2]}_i}\frac{\partial A^{[2]}_i}{\partial Z^{[2]}_j}\\
&=-\sum_i\frac{Y_i}{A^{[2]}_i} [A^{[2]}_i(1-A^{[2]}_i)\delta_{ij}-A^{[2]}_iA^{[2]}_j(1-\delta_{ij})]\\
&=-\sum_i{Y_i} [(1-A^{[2]}_i)\delta_{ij}-A^{[2]}_j(1-\delta_{ij})]\\
&=-{Y_j}(1-A^{[2]}_j) + A^{[2]}_j - {Y_j}A^{[2]}_j\\
&=-{Y_j}+ A^{[2]}_j
\end{align}
(Note that $\sum {Y_i}=1$)
so $\delta Z^{[2]}=A^{[2]} -Y$.

---

> Questions: Why we average over the training data to build $\delta W^{[2]}$ here?


\begin{align}
\frac{\partial Z^{[2]}_i}{\partial W^{[2]}_{ij}}&=A^{[1]}_j\\
\frac{\partial\mathcal{L}}{\partial W^{[2]}_{ij}}&=\frac{\partial\mathcal{L}}{\partial Z^{[2]}_i}\frac{\partial Z^{[2]}_i}{\partial W^{[2]}_{ij}}\\
&=(-{Y_i}+ A^{[2]}_i)\cdot A^{[1]}_j
\end{align}
so $\delta W^{[2]}=\frac{1}{m} \delta Z^{[2]}\cdot A^{[1]T}$

\begin{align}
\frac{\partial Z^{[2]}_i}{\partial B^{[2]}_{j}}&=\delta_{ij}\\
\frac{\partial\mathcal{L}}{\partial B^{[2]}_{i}}&=\frac{\partial\mathcal{L}}{\partial Z^{[2]}_i}\frac{\partial Z^{[2]}_i}{\partial B^{[2]}_{i}}\\
&=-{Y_i}+ A^{[2]}_i
\end{align}
so $\delta B^{[2]}=\frac{1}{m}\sum \delta Z^{[2]}$.


---
\begin{align}
\frac{\partial Z^{[2]}_j}{\partial A^{[1]}_k}&=W^{[2]}_{jk}\\
\frac{\partial A^{[1]}_k}{\partial Z^{[1]}_k}&=\Theta(Z^{[1]}_k)\\
\end{align}

\begin{align}
\frac{\partial\mathcal{L}}{\partial Z^{[1]}_k}&=\sum_{j}\frac{\partial\mathcal{L}}{\partial Z^{[2]}_j}\frac{\partial Z^{[2]}_j}{\partial A^{[1]}_k}\frac{\partial A^{[1]}_k}{\partial Z^{[1]}_k}\\
&=\sum_{j}(-{Y_j}+ A^{[2]}_j)W^{[2]}_{jk}\Theta(Z^{[1]}_k)
\end{align}
Note that $\sum_j$ is inner product and the $k$ index is element-wise muliplication. So $\delta Z^{[1]} = W^{[2]T}\cdot\delta Z^{[2]} * \Theta(Z^{[1]})$


Since $Z$ in the first second layers are both linear, we have $\delta W^{[1]}=\frac{1}{m} \delta Z^{[1]}\cdot A^{[0]T}$ where $A^{[0]}\equiv X$ and $\delta B^{[1]}=\frac{1}{m}\sum \delta Z^{[1]}$.

In [ ]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [ ]:
# test
W1, b1, W2, b2 = init_params()
Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X_train)
dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X_train, Y_train)

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

Iteration:  0
[3 1 7 ... 7 7 1] [7 4 0 ... 9 1 3]
0.08337280909521554
Iteration:  10
[4 1 3 ... 7 8 8] [7 4 0 ... 9 1 3]
0.1416916679825254
Iteration:  20
[4 9 3 ... 9 4 8] [7 4 0 ... 9 1 3]
0.1966945628717301
Iteration:  30
[4 9 1 ... 9 4 8] [7 4 0 ... 9 1 3]
0.26522448549923683
Iteration:  40
[4 9 6 ... 9 4 8] [7 4 0 ... 9 1 3]
0.3274909205747671
Iteration:  50
[2 9 6 ... 9 1 8] [7 4 0 ... 9 1 3]
0.373651244802358
Iteration:  60
[2 9 6 ... 7 1 8] [7 4 0 ... 9 1 3]
0.41712721722195906
Iteration:  70
[2 9 6 ... 7 1 8] [7 4 0 ... 9 1 3]
0.4640244223380178
Iteration:  80
[2 9 6 ... 7 1 8] [7 4 0 ... 9 1 3]
0.50734249171009
Iteration:  90
[2 9 6 ... 7 1 8] [7 4 0 ... 9 1 3]
0.5419758934680773
Iteration:  100
[7 9 6 ... 7 1 3] [7 4 0 ... 9 1 3]
0.5727669877361966
Iteration:  110
[7 9 6 ... 7 1 3] [7 4 0 ... 9 1 3]
0.5994526027685667
Iteration:  120
[7 9 6 ... 7 1 3] [7 4 0 ... 9 1 3]
0.6245591873256487
Iteration:  130
[7 9 6 ... 7 1 3] [7 4 0 ... 9 1 3]
0.6463498078846255
Iteration:  140
[